In [ ]:
### Mount google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import shutil
import zipfile
import urllib.request as request
from PIL import Image
from tqdm import tqdm

In [ ]:
### Download and extract attribute from single subject of humbi dataset

# attribute : attribute of interest (string)
# subject : subject of interest (integer)
# target_dir : path to which the data will be downloaded to (string)

# Return True if and only if download succeeded

def load_part(attribute, subject,
              root_url = 'https://humbi-dataset.s3.amazonaws.com',
              target_dir = os.getcwd()):
    part_name = attribute + '_subject'
    zip_file = 'subject_%d.zip' % subject # OR 'subject_' + str(subject) + '.zip'
    url = os.path.join(root_url, part_name, zip_file)
    target_path = os.path.join('%s_%d.zip' % (part_name, subject))

    try:
        request.urlretrieve(url, target_path) # OR !wget url
        downloaded_zip = zipfile.ZipFile(target_path)
        downloaded_zip.extractall(target_dir) # !unzip downloaded_zip
        return True
    except request.HTTPError:
        print(' %s attribute is missing for subject %d' % (attribute, subject))
        return False

In [ ]:
### Extract map from a subject

# attribute : attribute of interest (string)
# subject : subject of interest (integer)
# map : map of interest, i.e. mean/median/std/var (string)
# pose : body pose the subject is taking
# target_dir : path to which the subject data has been downloaded to (string)

# Return extracted image

def extract_image(attribute, subject, map, pose = '00000001', target_dir = os.getcwd()):
    assert(map in ['mean', 'median', 'std', 'var']), "map needs to be one of ['mean', 'median', 'std', 'var']"

    path_to_subject = os.path.join(target_dir, 'subject_%d' % subject)
    map_name = map + '_map' + '_hot' * (map == 'std' or map == 'var') + '.png'

    if attribute in ['body', 'body_texture', 'cloth']:
        attribute_path = 'body'
    elif attribute in ['face', 'face_texture']:
        attribute_path = 'face'
    elif attribute in ['gaze', 'gaze_texture']:
        attribute_path = 'gaze'
    else:
        attribute_path = attribute

    poses_path = os.path.join(path_to_subject, attribute_path)
    if pose not in os.listdir(poses_path):
        new_pose = sorted(os.listdir(poses_path))[0]
        print('\n pose %s not available for subject %d, we replace with first pose available : %s' % (pose, subject, new_pose))
        pose = new_pose

    image_path = os.path.join(path_to_subject, attribute_path, pose, 'appearance', map_name)
    image = Image.open(image_path)

    return image

In [ ]:
### Store image to drive

# image : PIL image to store
# save_path : path to which the image will be stored
# attribute : attribute of interest (string)
# subject : subject of interest (integer)
# map : map of interest, i.e. mean/median/std/var (string)

def save_to_drive(image, save_path, attribute, subject, map):
    assert(map in ['mean', 'median', 'std', 'var']), "map needs to be one of ['mean', 'median', 'std', 'var']"

    outer_directory = 'humbi_' + attribute
    inner_directory = attribute + '_' + map + 's'
    filename = map + '_subject_%d.png' % subject

    store_path = os.path.join(save_path, outer_directory, inner_directory, filename)
    os.makedirs(os.path.dirname(store_path), exist_ok = True)

    image.save(store_path, 'PNG')

    return

In [ ]:
### Clean disk

# attribute : attribute of interest (string)
# subject : subject of interest (integer)
# target_dir : path to which the subject data has been downloaded to (string)

def remove_subject(attribute, subject, target_dir = os.getcwd()):
    subject_directory = os.path.join(target_dir, 'subject_%d' % subject)
    subject_zip = os.path.join(target_dir, attribute + '_subject_%d.zip' % subject)

    shutil.rmtree(subject_directory)
    os.remove(subject_zip)

    return

In [ ]:
### Download humbi dataset

# attributes : list of body attributes/parts to load from ['body', 'body_texture', 'face', 'face_texture', 'gaze', 'gaze_texture', 'cloth', 'hand']
# subject_ids : range(1, 617)
# maps : list of maps to download from ['mean', 'median', 'std', 'var']
# poses : list of poses to download (start with default pose and see in the downloaded subject directorie which poses are available per subject)
# save_path : path to which the extracted maps will be stored
# target_dir : path to which the data will be downloaded to (string)

## CAREFUL : subjects are missing ; 'hand' attribute only has 453 subjects

ATTRIBUTES = ['body', 'body_texture', 'face', 'face_texture', 'gaze', 'gaze_texture', 'cloth', 'hand']
SUBJECT_IDS = range(1, 617)  # all subjects by default
MAPS = ['mean', 'median', 'std', 'var']
POSES = ['00000001'] # default pose
SAVE_PATH = '/content/drive/MyDrive/term_paper/humbi_maps'

def download_humbi_maps(attributes = ATTRIBUTES, subject_ids = SUBJECT_IDS,
                        maps = MAPS, poses = POSES, save_path = SAVE_PATH, 
                        root_url = 'https://humbi-dataset.s3.amazonaws.com',
                        target_dir = os.getcwd()):
    if not isinstance(attributes, list):
        attributes = [attributes]

    if isinstance(subject_ids, int):
        subject_ids = range(subject_ids, subject_ids + 1)

    if not isinstance(maps, list):
        maps = [attributes]

    if not isinstance(poses, list):
        poses = [poses]

    if not os.path.exists(target_dir):
        os.makedirs(target_dir)

    for subject in tqdm(subject_ids):
        for attribute in attributes:
            loaded = load_part(attribute, subject, root_url, target_dir)
            if loaded:
                for map in maps:
                    for pose in poses:
                        image = extract_image(attribute, subject, map, pose, target_dir)
                        save_to_drive(image, save_path, attribute, subject, map)
                remove_subject(attribute, subject, target_dir)

    return

In [ ]:
download_humbi_maps(attributes = 'body_texture')

In [ ]:
### See https://github.com/lucidrains/lightweight-gan
!pip install lightweight-gan

In [ ]:
path_to_drive = '/content/drive/MyDrive/term_paper'

training_data = path_to_drive + '/humbi_maps/humbi_body_texture/body_texture_medians/'
models_dir = path_to_drive + '/generated_maps/model/'
results_dir = path_to_drive + '/generated_maps/generated_images/'

run_name = 'sample_run_256'

In [ ]:
### Training
!lightweight_gan  --data {training_data} \
                  --name {run_name} \
                  --models-dir {models_dir} \
                  --results-dir {results_dir} \
                  --save-every 1000 \
                  --batch-size 8 \
                  --gradient-accumulate-every 4 \
                  --num-train-steps 200000 \
                  --image-size 256

In [ ]:
### Generating
!lightweight_gan  --name {run_name} \
                  --models-dir {models_dir} \
                  --results-dir {results_dir} \
                  --generate \
                  --generate-types default \
                  --num-image-tiles 16
                  # --load-from {checkpoint num}

In [ ]:
### Show progress
!lightweight_gan  --name {run_name} \
                  --models-dir {models_dir} \
                  --results-dir {results_dir} \
                  --show-progress \
                  --generate-types default \
                  --num-image-tiles 1

In [ ]:
### Progress to video
progress_imgs = path_to_drive + '/generated_maps/generated_images/' + run_name + '-progress/*.jpg'
progress_video = path_to_drive + '/generated_maps/generated_images/' + run_name + '.mp4'

!ffmpeg -framerate 10 -pattern_type glob -i '{progress_imgs}' {progress_video}
print('Video created and saved under', progress_video)